This training process is slightly more complex since we are sampling from our training set and generating adversarial examples on the fly. Still, the benefit is that the model can:

- Learn patterns from the original training set
- Learn patterns from the adversarial examples
- Since the model has now been trained on adversarial examples, it will be more robust and generalize better when presented with adversarial images.

In [20]:
#---Utilities
from pyimagesearch.simplecnn import SimpleCNN 
from pyimagesearch.datagen_mix import generate_mixed_adversarial_batch
from pyimagesearch.datagen import generate_adversarial_batch
#---Tensorflow
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
#---Others
import numpy as np

In [2]:
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

In [3]:
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

In [4]:
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

In [5]:
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=1e-3), metrics=["accuracy"])

2022-09-26 16:52:26.622218: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-09-26 16:52:26.622268: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: juan-GS63VR-7RF
2022-09-26 16:52:26.622284: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: juan-GS63VR-7RF
2022-09-26 16:52:26.622394: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-09-26 16:52:26.622435: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.65.1
2022-09-26 16:52:26.622449: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.65.1
2022-09-26 16:52:26.622989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performan

In [7]:
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=10, verbose=1)

Epoch 1/10
938/938 [==============================] - 10s 11ms/step - loss: 0.0278 - accuracy: 0.9909 - val_loss: 0.0479 - val_accuracy: 0.9858
Epoch 2/10
938/938 [==============================] - 10s 11ms/step - loss: 0.0249 - accuracy: 0.9919 - val_loss: 0.0419 - val_accuracy: 0.9876
Epoch 3/10
938/938 [==============================] - 11s 12ms/step - loss: 0.0221 - accuracy: 0.9929 - val_loss: 0.0371 - val_accuracy: 0.9876
Epoch 4/10
938/938 [==============================] - 11s 12ms/step - loss: 0.0180 - accuracy: 0.9941 - val_loss: 0.0415 - val_accuracy: 0.9879
Epoch 5/10
938/938 [==============================] - 11s 12ms/step - loss: 0.0173 - accuracy: 0.9941 - val_loss: 0.0411 - val_accuracy: 0.9889
Epoch 6/10
938/938 [==============================] - 12s 12ms/step - loss: 0.0162 - accuracy: 0.9945 - val_loss: 0.0536 - val_accuracy: 0.9871
Epoch 7/10
938/938 [==============================] - 11s 12ms/step - loss: 0.0150 - accuracy: 0.9951 - val_loss: 0.0413 - val_accuracy:

In [11]:
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print("Loss: {:.4f}, acc: {:.4f}".format(loss, acc))

Loss: 0.0414, acc: 0.9887


In [22]:
(advX, advY) = next(generate_adversarial_batch(model, len(testX), testX, testY, (28, 28, 1), eps=0.1))

In [13]:
(loss, acc) = model.evaluate(x=advX, y=advY, verbose=0)
print("Loss: {:.4f}, acc: {:.4f}".format(loss, acc))

Loss: 27.3050, acc: 0.0000


### Re-tune the model

In [14]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=1e-4), metrics=["accuracy"])

### Use the mix adversarial dataset

In [15]:
dataGen = generate_mixed_adversarial_batch(model, 64, trainX, trainY, (28, 28, 1), eps=0.1, split=0.5)

In [16]:
#Add steps per epoch to deal with the generator
model.fit(dataGen, steps_per_epoch=len(trainX) // 64, epochs=10, verbose=1)

Epoch 1/10
937/937 [==============================] - 218s 233ms/step - loss: 0.2393 - accuracy: 0.9687
Epoch 2/10
937/937 [==============================] - 214s 229ms/step - loss: 0.1142 - accuracy: 0.9747
Epoch 3/10
937/937 [==============================] - 215s 229ms/step - loss: 0.0744 - accuracy: 0.9799
Epoch 4/10
937/937 [==============================] - 214s 229ms/step - loss: 0.0544 - accuracy: 0.9834
Epoch 5/10
937/937 [==============================] - 217s 232ms/step - loss: 0.0559 - accuracy: 0.9837
Epoch 6/10
937/937 [==============================] - 213s 228ms/step - loss: 0.0458 - accuracy: 0.9860
Epoch 7/10
937/937 [==============================] - 218s 232ms/step - loss: 0.0420 - accuracy: 0.9868
Epoch 8/10
937/937 [==============================] - 216s 231ms/step - loss: 0.0396 - accuracy: 0.9867
Epoch 9/10
937/937 [==============================] - 214s 229ms/step - loss: 0.0378 - accuracy: 0.9886
Epoch 10/10
937/937 [==============================] - 215s 229m

In [17]:
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print("Normal testing images *after* fine-tuning:")
print("Loss: {:.4f}, acc: {:.4f}\n".format(loss, acc))

Normal testing images *after* fine-tuning:
Loss: 0.0323, acc: 0.9908

